# Loop 2 LB Feedback Analysis

## Situation
- Both submissions scored exactly 70.7343 (CV = LB perfectly)
- Extended optimization (45 min) found only 3 tiny improvements that don't show in rounded score
- The santa-2025.csv starting submission is already at a very strong local optimum
- Gap to target: 1.803 points (2.55%)

In [1]:
import pandas as pd
import numpy as np
import os

# Load the current best submission
submission_path = '/home/code/datasets/santa-2025-csv/santa-2025.csv'
df = pd.read_csv(submission_path)
print(f"Loaded {len(df)} rows")
print(df.head())

Loaded 20100 rows
      id                       x                        y  \
0  001_0  s43.591192092102147626  s-31.783267068741778871   
1  002_0   s0.154097069621360605   s-0.038540742694777107   
2  002_1  s-0.154097069621359162   s-0.561459257305227277   
3  003_0   s1.131270585068746337    s0.792202872326948637   
4  003_1   s1.234055695842160016    s1.275999500663759001   

                       deg  
0   s44.999999999999978684  
1  s203.629377730650162448  
2   s23.629377730649704148  
3  s113.563260441729482864  
4   s66.370622269343002131  


In [2]:
# Calculate per-N scores
def strip_s(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

# Tree polygon vertices
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_bbox(x, y, deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    px = TX * c - TY * s + x
    py = TX * s + TY * c + y
    return px.min(), py.min(), px.max(), py.max()

def calc_n_score(group):
    n = len(group)
    minx, miny, maxx, maxy = np.inf, np.inf, -np.inf, -np.inf
    for _, row in group.iterrows():
        x = strip_s(row['x'])
        y = strip_s(row['y'])
        deg = strip_s(row['deg'])
        x0, y0, x1, y1 = get_tree_bbox(x, y, deg)
        minx = min(minx, x0)
        miny = min(miny, y0)
        maxx = max(maxx, x1)
        maxy = max(maxy, y1)
    side = max(maxx - minx, maxy - miny)
    return side, side**2 / n

# Calculate scores for each N
df['N'] = df['id'].str.split('_').str[0].astype(int)
scores = []
for n, group in df.groupby('N'):
    side, score = calc_n_score(group)
    scores.append({'N': n, 'side': side, 'score': score})

scores_df = pd.DataFrame(scores)
print(f"Total score: {scores_df['score'].sum():.6f}")
print(f"\nWorst 20 N values by score contribution:")
print(scores_df.nlargest(20, 'score')[['N', 'side', 'score']])

Total score: 70.734327

Worst 20 N values by score contribution:
     N      side     score
0    1  0.813173  0.661250
1    2  0.949504  0.450779
2    3  1.142031  0.434745
4    5  1.443692  0.416850
3    4  1.290806  0.416545
6    7  1.673104  0.399897
5    6  1.548438  0.399610
8    9  1.867280  0.387415
7    8  1.755921  0.385407
14  15  2.384962  0.379203
9   10  1.940696  0.376630
20  21  2.811667  0.376451
19  20  2.742469  0.376057
10  11  2.033002  0.375736
21  22  2.873270  0.375258
15  16  2.446640  0.374128
25  26  3.118320  0.373997
11  12  2.114873  0.372724
12  13  2.200046  0.372323
24  25  3.050182  0.372144


In [3]:
# Calculate efficiency (how close to theoretical optimal)
# Theoretical optimal for N trees: side = sqrt(N * tree_area)
# Tree area is approximately 0.35 * 0.8 = 0.28 (rough estimate)
tree_area = 0.28  # approximate

scores_df['theoretical_side'] = np.sqrt(scores_df['N'] * tree_area)
scores_df['efficiency'] = scores_df['theoretical_side'] / scores_df['side']

print("Efficiency analysis (higher is better, max 1.0):")
print(f"Mean efficiency: {scores_df['efficiency'].mean():.4f}")
print(f"Min efficiency: {scores_df['efficiency'].min():.4f} at N={scores_df.loc[scores_df['efficiency'].idxmin(), 'N']}")
print(f"Max efficiency: {scores_df['efficiency'].max():.4f} at N={scores_df.loc[scores_df['efficiency'].idxmax(), 'N']}")

print("\nWorst 20 N values by efficiency:")
print(scores_df.nsmallest(20, 'efficiency')[['N', 'side', 'score', 'efficiency']])

Efficiency analysis (higher is better, max 1.0):
Mean efficiency: 0.8913
Min efficiency: 0.6507 at N=1
Max efficiency: 0.9212 at N=181

Worst 20 N values by efficiency:
     N      side     score  efficiency
0    1  0.813173  0.661250    0.650723
1    2  0.949504  0.450779    0.788129
2    3  1.142031  0.434745    0.802531
4    5  1.443692  0.416850    0.819576
3    4  1.290806  0.416545    0.819876
6    7  1.673104  0.399897    0.836768
5    6  1.548438  0.399610    0.837068
8    9  1.867280  0.387415    0.850141
7    8  1.755921  0.385407    0.852352
14  15  2.384962  0.379203    0.859297
9   10  1.940696  0.376630    0.862227
20  21  2.811667  0.376451    0.862432
19  20  2.742469  0.376057    0.862884
10  11  2.033002  0.375736    0.863252
21  22  2.873270  0.375258    0.863802
15  16  2.446640  0.374128    0.865105
25  26  3.118320  0.373997    0.865257
11  12  2.114873  0.372724    0.866733
12  13  2.200046  0.372323    0.867199
24  25  3.050182  0.372144    0.867408


In [4]:
# Gap analysis
target = 68.931058
current = scores_df['score'].sum()
gap = current - target

print(f"Current score: {current:.6f}")
print(f"Target score: {target:.6f}")
print(f"Gap: {gap:.6f} ({gap/current*100:.2f}%)")

# How much improvement needed per N on average?
print(f"\nAverage improvement needed per N: {gap/200:.6f}")
print(f"Average % improvement needed: {gap/current*100:.2f}%")

# If we improve worst 20 N values by X%, what's the impact?
worst_20_score = scores_df.nlargest(20, 'score')['score'].sum()
print(f"\nWorst 20 N values contribute: {worst_20_score:.4f} ({worst_20_score/current*100:.2f}% of total)")
print(f"To close gap by improving worst 20: need {gap/worst_20_score*100:.2f}% improvement on worst 20")

Current score: 70.734327
Target score: 68.931058
Gap: 1.803269 (2.55%)

Average improvement needed per N: 0.009016
Average % improvement needed: 2.55%

Worst 20 N values contribute: 8.0771 (11.42% of total)
To close gap by improving worst 20: need 22.33% improvement on worst 20


In [5]:
# Analyze what the egortrushin kernel does differently
# Key insight: For large N (>=58), use periodic/crystalline structures
# This means arranging trees in a regular grid pattern with unit cells

print("Key insight from egortrushin kernel:")
print("="*50)
print("For N >= 58, use PERIODIC STRUCTURES:")
print("- Define a 'unit cell' of trees")
print("- Tile the unit cell to create the full configuration")
print("- Optimize the unit cell parameters (translations, rotations)")
print("- This can find configurations that standard SA cannot reach")
print()
print("The approach uses:")
print("1. nt = [rows, cols] - number of unit cells in each direction")
print("2. Translations determined via SA updates")
print("3. All trees in unit cell rotated by same angle")
print("4. Option to translate only one tree during last translation")
print()
print("This is fundamentally different from our current approach!")
print("Current approach: optimize each tree independently")
print("Periodic approach: optimize unit cell, then tile")

Key insight from egortrushin kernel:
For N >= 58, use PERIODIC STRUCTURES:
- Define a 'unit cell' of trees
- Tile the unit cell to create the full configuration
- Optimize the unit cell parameters (translations, rotations)
- This can find configurations that standard SA cannot reach

The approach uses:
1. nt = [rows, cols] - number of unit cells in each direction
2. Translations determined via SA updates
3. All trees in unit cell rotated by same angle
4. Option to translate only one tree during last translation

This is fundamentally different from our current approach!
Current approach: optimize each tree independently
Periodic approach: optimize unit cell, then tile


In [6]:
# Check which N values might benefit from periodic structures
print("N values that might benefit from periodic structures (N >= 58):")
large_n = scores_df[scores_df['N'] >= 58]
print(f"Number of large N values: {len(large_n)}")
print(f"Score contribution from large N: {large_n['score'].sum():.4f} ({large_n['score'].sum()/current*100:.2f}%)")

# Factorization analysis - which N values have nice factorizations?
print("\nN values with nice factorizations (for periodic structures):")
for n in range(58, 201):
    factors = []
    for i in range(2, int(np.sqrt(n)) + 1):
        if n % i == 0:
            factors.append((i, n // i))
    if factors:
        print(f"N={n}: {factors}")

N values that might benefit from periodic structures (N >= 58):
Number of large N values: 143
Score contribution from large N: 49.1728 (69.52%)

N values with nice factorizations (for periodic structures):
N=58: [(2, 29)]
N=60: [(2, 30), (3, 20), (4, 15), (5, 12), (6, 10)]
N=62: [(2, 31)]
N=63: [(3, 21), (7, 9)]
N=64: [(2, 32), (4, 16), (8, 8)]
N=65: [(5, 13)]
N=66: [(2, 33), (3, 22), (6, 11)]
N=68: [(2, 34), (4, 17)]
N=69: [(3, 23)]
N=70: [(2, 35), (5, 14), (7, 10)]
N=72: [(2, 36), (3, 24), (4, 18), (6, 12), (8, 9)]
N=74: [(2, 37)]
N=75: [(3, 25), (5, 15)]
N=76: [(2, 38), (4, 19)]
N=77: [(7, 11)]
N=78: [(2, 39), (3, 26), (6, 13)]
N=80: [(2, 40), (4, 20), (5, 16), (8, 10)]
N=81: [(3, 27), (9, 9)]
N=82: [(2, 41)]
N=84: [(2, 42), (3, 28), (4, 21), (6, 14), (7, 12)]
N=85: [(5, 17)]
N=86: [(2, 43)]
N=87: [(3, 29)]
N=88: [(2, 44), (4, 22), (8, 11)]
N=90: [(2, 45), (3, 30), (5, 18), (6, 15), (9, 10)]
N=91: [(7, 13)]
N=92: [(2, 46), (4, 23)]
N=93: [(3, 31)]
N=94: [(2, 47)]
N=95: [(5, 19)]
N=9

In [7]:
# Summary of key findings
print("="*60)
print("KEY FINDINGS FOR NEXT EXPERIMENT")
print("="*60)
print()
print("1. CURRENT STATE:")
print(f"   - Score: 70.7343 (gap to target: 1.803 = 2.55%)")
print(f"   - Standard SA/local search cannot improve the score")
print(f"   - The starting submission is at a very strong local optimum")
print()
print("2. STRATEGIC PIVOT REQUIRED:")
print("   - Implement PERIODIC STRUCTURE optimization (egortrushin approach)")
print("   - For N >= 58, use unit cell tiling instead of individual tree optimization")
print("   - This explores a fundamentally different part of the solution space")
print()
print("3. SPECIFIC ACTIONS:")
print("   a) Implement periodic SA from egortrushin kernel")
print("   b) Focus on N values with nice factorizations (60, 64, 72, 80, etc.)")
print("   c) Run for MUCH longer (hours, not minutes)")
print("   d) Try multiple random seeds to escape local optima")
print()
print("4. EXPECTED IMPACT:")
print(f"   - Large N (>=58) contributes {large_n['score'].sum():.2f} to total score")
print(f"   - Even 5% improvement on large N would save {large_n['score'].sum()*0.05:.2f} points")
print(f"   - Combined with small improvements elsewhere, target is achievable")

KEY FINDINGS FOR NEXT EXPERIMENT

1. CURRENT STATE:
   - Score: 70.7343 (gap to target: 1.803 = 2.55%)
   - Standard SA/local search cannot improve the score
   - The starting submission is at a very strong local optimum

2. STRATEGIC PIVOT REQUIRED:
   - Implement PERIODIC STRUCTURE optimization (egortrushin approach)
   - For N >= 58, use unit cell tiling instead of individual tree optimization
   - This explores a fundamentally different part of the solution space

3. SPECIFIC ACTIONS:
   a) Implement periodic SA from egortrushin kernel
   b) Focus on N values with nice factorizations (60, 64, 72, 80, etc.)
   c) Run for MUCH longer (hours, not minutes)
   d) Try multiple random seeds to escape local optima

4. EXPECTED IMPACT:
   - Large N (>=58) contributes 49.17 to total score
   - Even 5% improvement on large N would save 2.46 points
   - Combined with small improvements elsewhere, target is achievable
